<a href="https://colab.research.google.com/github/NeuralDataMind/PyTorch/blob/main/Logistic_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import torch
import torch.nn as nn

import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

In [63]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [64]:
X, y = make_classification(
    n_samples = 1000,
    n_features = 17 ,
    n_informative = 10,
    n_redundant = 7,
    n_classes = 2,
    random_state = 21
)

In [65]:
X.shape

(1000, 17)

In [66]:
n_samples, n_features = X.shape

In [67]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 18)

In [68]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [69]:
X_train = torch.from_numpy(X_train).type(torch.float32).to(device)
X_test = torch.from_numpy(X_test).type(torch.float32).to(device)
y_train = torch.from_numpy(y_train).type(torch.float32).view(-1, 1).to(device)
y_test = torch.from_numpy(y_test).type(torch.float32).view(-1, 1).to(device)

In [70]:
class LogisticRegression(nn.Module):
  def __init__(self, n_input_features):
    super(LogisticRegression, self).__init__()
    self.linear_1 = nn.Linear(n_input_features, 20)
    self.linear_2 = nn.Linear(20, 1)
    self.elu = nn.ELU()

  def forward(self, x):
    x = self.elu(self.linear_1(x))
    y_predicted = torch.sigmoid(self.linear_2(x))
    return y_predicted

In [71]:
model = LogisticRegression(n_features).to(device)

In [72]:
learning_rate = 0.1
criterion = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)

In [73]:
num_epochs = 6500

for epoch in range(num_epochs):
  model.train()

  y_predicted = model(X_train)
  loss = criterion(y_predicted, y_train)

  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

  if (epoch + 1) % 500 == 0:
    print(f'epoch {epoch + 1}, loss = {loss.item():.2f}')

epoch 500, loss = 0.31
epoch 1000, loss = 0.25
epoch 1500, loss = 0.21
epoch 2000, loss = 0.18
epoch 2500, loss = 0.16
epoch 3000, loss = 0.14
epoch 3500, loss = 0.13
epoch 4000, loss = 0.12
epoch 4500, loss = 0.11
epoch 5000, loss = 0.11
epoch 5500, loss = 0.10
epoch 6000, loss = 0.10
epoch 6500, loss = 0.09


In [74]:
model.eval()

with torch.inference_mode():
  y_prediceted = model(X_test)
  y_predicted_cls = y_prediceted.round()
  acc = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
  print(f'accuracy = {acc:.2f}')

accuracy = 0.91
